In [2]:
import json
import pandas as pd
import re
from collections import defaultdict, Counter
from bs4 import BeautifulSoup

# 读取数据集
# LABEL_PATH = './data/2023S-T1-A-Data[供选手] 0513/指令数据&标签/'
LABEL_PATH = './data/2023S-T1-A-Data[供选手] 0513/指令数据&标签/'
label_trainset = json.load(open(LABEL_PATH + 'label_trainset.json'))
instruction_trainset = json.load(open(LABEL_PATH + 'instruction_trainset.json'))
instruction_testA = json.load(open(LABEL_PATH + 'instruction_testA.json'))
instruction_JCZX = json.load(open(LABEL_PATH + 'instruction_JCZX.json'))
instruction_testB = json.load(open('./data/instruction_testB_with_node.json'))

In [28]:
import random

chat_data = []
for idx in range(20):
    for instructions in instruction_trainset[idx]['instruction_detail']:
        chat_data.append({
            'prompt': instructions['instruction'],
            'response': ';'.join([f'{key}' for key, value in instructions['key-value'].items()]),
            "history": []
        })
random.shuffle(chat_data)

In [2]:
# 对指令instructors进行分词
import jieba

def tokenized(f):
    text = "/".join(jieba.cut(f, cut_all=False, HMM=True)) # 精确模式
    return text


In [30]:
chat_labeled_data = []
prompt = "根据用户在两个#之间输入的自然语言指令, 并参考/的分词结果,在网页中规划（输出）一条动作序列。#{0}#"
for idx in range(20):
    for instructions in label_trainset[idx]['instruction_detail']:
        chat_labeled_data.append({
            'prompt': prompt.format(tokenized(instructions['instruction'])),
            'response': ';'.join(["{}#{}*{}*{}#".format(key, value['dom_type'], value['value'], value['action']) for key, value in instructions['key-value'].items()]),
            "history": []
        })
        
# 金融数据补充（BC数据集）
for bc_instructions in instruction_JCZX:
    chat_labeled_data.append({
            'prompt': prompt.format(tokenized(bc_instructions['instruction'])),
            'response': ';'.join(["{}#{}*{}*{}#".format(key, value['dom_type'], value['value'], value['action']) for key, value in bc_instructions['key-value'].items()]),
            "history": []
        })

random.shuffle(chat_labeled_data)

In [31]:
chat_labeled_data[2000]

{'prompt': '根据用户在两个#之间输入的自然语言指令, 并参考/的分词结果,在网页中规划（输出）一条动作序列。#请/搜索/半淞/园地/段/医院/更新/时间/为/2018/年/1/月/1/日到/2023/年/3/月/1/日/的/女护士/吴玮/的/信息#',
 'response': '查询#按钮**点击#;护士姓名#输入框*吴玮*输入#;性别#下拉框**点击#;女#下拉框选项**点击#;执业地点#输入框*半淞园地段医院*输入#;更新时间_1#日期输入框*2018-01-01*输入#;更新时间_2#日期输入框*2023-03-01*输入#',
 'history': []}

In [32]:
chat_labeled_data[0]

{'prompt': '根据用户在两个#之间输入的自然语言指令, 并参考/的分词结果,在网页中规划（输出）一条动作序列。#请/查找/港台地区/的/男性/的/流行/风格/的/歌手/列表#',
 'response': '搜索#按钮**点击#;港台#单选框**点击#;男#单选框**点击#;流行#单选框**点击#',
 'history': []}

In [33]:
chat_labeled_data[2398:2402]

[{'prompt': '根据用户在两个#之间输入的自然语言指令, 并参考/的分词结果,在网页中规划（输出）一条动作序列。#查查/深振业/A/最近/三年/三板/的/中介机构/公告#',
  'response': '查询#按钮**点击#;深振业A#输入框**输入#;2020-03-22#日期输入框**输入#;2023-03-22#日期输入框**输入#;三板#按钮**点击#;分类#下拉框**点击#;中介机构公告#下拉框选项**点击#;确认#按钮**点击#',
  'history': []},
 {'prompt': '根据用户在两个#之间输入的自然语言指令, 并参考/的分词结果,在网页中规划（输出）一条动作序列。#请查/一下/包含/以下/全部/的/关键词/如/”/环境/资源/“/,/包含/以下/任意/一个/关键词/如/”/经济核算/“/,/不/包含/以下/关键词/如/”/地方/自治/“/,/2022/-/01/-/01/到/2023/-/01/-/01/的/查询/结果#',
  'response': '开始搜索#按钮**点击#;包含以下全部的关键词#输入框*环境资源*输入#;包含以下任意一个关键词#输入框*经济核算*输入#;不包含以下关键词#输入框*地方自治*输入#;自定义时间_1#日期输入框*2022-01-01*输入#;自定义时间_2#日期输入框*2023-01-01*输入#',
  'history': []},
 {'prompt': '根据用户在两个#之间输入的自然语言指令, 并参考/的分词结果,在网页中规划（输出）一条动作序列。#请/查询/属于/分公司/层级/,/受/监管/于/秦皇岛/银保监/分局/,/受/监管/于/浙江/银/保监局/,/广西壮族自治区/南宁市/青秀区/桂春路/13/号/,/机构/类型/为/政策性/保险公司/,/综合/产险/公司/类别/,/机构/名为/都/邦/财产保险/股份/有限公司/北仑/支公司/,/流水号/00009243/的/机构/信息#',
  'response': '查询#按钮**点击#;机构名称#输入框*都邦财产保险股份有限公司北仑支公司*输入#;机构地址#输入框*广西壮族自治区南宁市青秀区桂春路13号*输入#;流水号#输入框*00009243*输入#;机构类型_1#下拉框**点击#;政策性保险公司#下拉框选项**点击#;

In [34]:
import numpy as np
np.max([len(x['response']) for x in chat_data])

71

In [35]:
import numpy as np
np.max([len(x['response']) for x in chat_labeled_data])

348

In [36]:
with open('train.json', 'w') as up:
    for line in chat_data[:-400]:
        up.write(json.dumps(line)+'\n')
    
with open('dev.json', 'w') as up:
    for line in chat_data[-400:]:
        up.write(json.dumps(line)+'\n')

In [37]:
with open('label_train.json', 'w') as up:
    for line in chat_labeled_data[:-400]:
        up.write(json.dumps(line)+'\n')
    
with open('label_dev.json', 'w') as up:
    for line in chat_labeled_data[-400:]:
        up.write(json.dumps(line)+'\n')

In [38]:
line

{'prompt': '根据用户在两个#之间输入的自然语言指令, 并参考/的分词结果,在网页中规划（输出）一条动作序列。#请/搜索/深沪京/的/可转债/和/股权/激励/公告#',
 'response': '查询#按钮**点击#;深沪京#按钮**点击#;分类#下拉框**点击#;可转债#下拉框选项**点击#;股权激励#下拉框选项**点击#;确认#按钮**点击#',
 'history': []}

In [39]:
instruction_trainset[-1]['instruction_detail'][-1]

{'instruction': '查询从上海市黄埔区寄到法国Haizhu的快递运费，其中寄件时间是2023-01-09上午08:20:11，包裹重量为0.8kg，目的地邮编为999004',
 'key-value': {'始发地': '上海市-上海市-黄埔区',
  '目的地': '法国-Haizhu',
  '寄件时间': '2023-01-09 08:20:11',
  '目的地邮编': '999004',
  '包裹重量': '0.800'}}

In [40]:
instruction_trainset[11]['instruction_detail'][11]

{'instruction': '查找企业状态为存续（在营、开业、在册）的企业信息', 'key-value': {'存续（在营、开业、在册）': ''}}

In [41]:
label_trainset[10]['instruction_detail'][10]

{'instruction': '请搜索上海豫园地段医院的护士信息',
 'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
  '执业地点': {'dom_type': '输入框', 'value': '上海豫园地段医院', 'action': '输入'}}}

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig

config = AutoConfig.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)
# config.pre_seq_len = 128
config.pre_seq_len = 256

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, config=config).half()
#prefix_state_dict = torch.load('./ChatGLM-6B/ptuning/output/adgen-chatglm-6b-pt-128-1e-2/checkpoint-800/pytorch_model.bin') score 54.9
# prefix_state_dict = torch.load('./ChatGLM-6B/ptuning/output/adgen-chatglm-6b-pt-128-2e-2/checkpoint-400/pytorch_model.bin') # score 61.83
# prefix_state_dict = torch.load('./ChatGLM-6B/ptuning/output/adgen-chatglm-6b-pt-256-2e-2/checkpoint-600/pytorch_model.bin') # score 66.1
prefix_state_dict = torch.load('./ChatGLM-6B/ptuning/output/adgen0526-chatglm-6b-pt-256-2e-2/checkpoint-600/pytorch_model.bin') # score 
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    if k.startswith("transformer.prefix_encoder."):
        new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)

model = model.quantize(4)
model = model.half().cuda()
model.transformer.prefix_encoder.float()
model = model.eval()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at THUDM/chatglm-6b and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
respons = model.chat(tokenizer,"根据用户在两个#之间输入的自然语言指令，在网页中规划（输出）一条动作序列。#查询从浙江省杭州市南沙区寄到新加坡Zengcheng的快递时效，其中寄件时间是2023-01-17下午12:23:45，包裹重量为1.7kg，目的地邮编为999005#", history=[])
print(respons)

The dtype of attention mask (torch.int64) is not bool


('查询#按钮**点击#；始发地#输入框*浙江省杭州市南沙区*输入#；目的地#输入框*新加坡Zengcheng*输入#；寄件时间#输入框*2023-01-17下午12:23:45*输入#；目的地邮编#输入框*999005*输入#；包裹重量#输入框*1.700*输入#', [('根据用户在两个#之间输入的自然语言指令，在网页中规划（输出）一条动作序列。#查询从浙江省杭州市南沙区寄到新加坡Zengcheng的快递时效，其中寄件时间是2023-01-17下午12:23:45，包裹重量为1.7kg，目的地邮编为999005#', '查询#按钮**点击#；始发地#输入框*浙江省杭州市南沙区*输入#；目的地#输入框*新加坡Zengcheng*输入#；寄件时间#输入框*2023-01-17下午12:23:45*输入#；目的地邮编#输入框*999005*输入#；包裹重量#输入框*1.700*输入#')])


In [5]:
bad_sample = "检索一下关于中金公司，2019年1月1日到2023年9月1日期间，当事人是陈义的律师是宗彦的通知书"
prompt_tmp = "根据用户在两个#之间输入的自然语言指令，并参考/的分词结果，在网页中规划（输出）一条动作序列。#{0}#"
respons = model.chat(tokenizer,prompt_tmp.format(tokenized(bad_sample)), history=[])
print(respons)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.664 seconds.
Prefix dict has been built successfully.


('检索#按钮**点击#；全文检索#输入框*中金公司*输入#；裁判日期_1#日期输入框*2019-01-01*输入#；裁判日期_2#日期输入框*2023-09-01*输入#；文书类型#下拉框**点击#；通知书#下拉框选项**点击#；当事人#输入框*陈义*输入#；律师#输入框*宗彦*输入#；高级检索#按钮**点击#', [('根据用户在两个#之间输入的自然语言指令，并参考/的分词结果，在网页中规划（输出）一条动作序列。#检索/一下/关于/中金公司/，/2019/年/1/月/1/日到/2023/年/9/月/1/日/期间/，/当事人/是/陈义/的/律师/是/宗彦/的/通知书#', '检索#按钮**点击#；全文检索#输入框*中金公司*输入#；裁判日期_1#日期输入框*2019-01-01*输入#；裁判日期_2#日期输入框*2023-09-01*输入#；文书类型#下拉框**点击#；通知书#下拉框选项**点击#；当事人#输入框*陈义*输入#；律师#输入框*宗彦*输入#；高级检索#按钮**点击#')])


In [ ]:
bad_sample = "查一查是否可以把毛毯从上海市黄埔区寄到美国Yuexiu，其中寄件人身份是公司，收件人身份是个人"
prompt_tmp = "根据用户在两个#之间输入的自然语言指令，并参考/的分词结果，在网页中规划（输出）一条动作序列。#{0}#"
respons = model.chat(tokenizer,prompt_tmp.format(tokenized(bad_sample)), history=[])
print(respons)

In [7]:
###### 对A榜数据进行预测
submit_predictions = []
pattern = r"(\w+)#(\w+)\*(.*?)\*(\w+)#"
prompt = "根据用户在两个#之间输入的自然语言指令，并参考/的分词结果，在网页中规划（输出）一条动作序列。#{0}#"


for idx in range(len(instruction_testA)):
    page_predictions = []
    for instruction_detail in instruction_testA[idx]['instruction_detail']: 
        prediction = {}
        data_dict = {}
        prediction['instruction'] = instruction_detail['instruction']
        pred_data = prompt.format(tokenized(instruction_detail['instruction']))
        model_out = model.chat(tokenizer,pred_data , history=[])
        output = model_out[0].replace(";", "；") 
        datas = output.split("；")
        for item in datas:
            try:
                matches = re.findall(pattern, item)
                key = matches[0][0]
                dom_type = matches[0][1]
                value = matches[0][2]
                action =matches[0][3]
                data_dict[key] = {"dom_type": dom_type, "value":value, "action": action}
                    
            except: 
                print("except: {}".format(model_out))
                data_dict[""] = {"dom_type": "", "value":"", "action":""}
                print("{}-{}-{}".format(instruction_testA[idx]['page_source'],instruction_detail['instruction'],  item))
                continue
                
                   
        prediction['key-value'] = data_dict         
        page_predictions.append(prediction)

        
                                                                     
    pag_source = instruction_testA[idx]['page_source']
    submit_predictions.append({
        'page_source': pag_source, 
        'instruction_detail': page_predictions
    })


# 生成提交结果
with open('submission.json', 'w') as up:
    json.dump(submit_predictions, up)
    

except: ('高级搜索#按钮**点击#；包含以下全部的关键词#输入框*货币*输入#；包含以下任意一个关键词#输入框*征信*输入#；不包含以下关键词#输入框*市场干扰*输入#；自定义时间_1#日期输入框*2022-05-01*输入#；自定义时间_2#日期输入框*2023-03-01*输入#；默认#单选框**点击#; within#单选框**点击#；请选择查询范围#下拉框**点击#;', [('根据用户在两个#之间输入的自然语言指令，并参考/的分词结果，在网页中规划（输出）一条动作序列。#帮/我查/一下/包含/全部/关键词/如/”/货币/“/,/包含/某个/关键词/如/”/征信/“/,/不/包含/关键词/如/”/市场/干扰/“/,/关键词/在/默认/位置/,/在/“/党委/宣传部/“/范围/内/查询/的/时间/不限/的/查询/结果#', '高级搜索#按钮**点击#；包含以下全部的关键词#输入框*货币*输入#；包含以下任意一个关键词#输入框*征信*输入#；不包含以下关键词#输入框*市场干扰*输入#；自定义时间_1#日期输入框*2022-05-01*输入#；自定义时间_2#日期输入框*2023-03-01*输入#；默认#单选框**点击#; within#单选框**点击#；请选择查询范围#下拉框**点击#;')])
中国人民银行-帮我查一下包含全部关键词如”货币“,包含某个关键词如”征信“,不包含关键词如”市场干扰“,关键词在默认位置,在“党委宣传部“范围内查询的时间不限的查询结果-
except: ('检索#按钮**点击#；全文检索#输入框*牛晨轩*输入#；法院名称#输入框*上海市金融法院*输入#；裁判日期_1#日期输入框*2022-10-15*输入#；裁判日期_2#日期输入框*2023-03-15*输入#；文书类型#下拉框**点击#；裁定书#下拉框选项**点击#；当事人#输入框*牛晨轩*输入#；律师#输入框*贺涵易*输入#；高级检索', [('根据用户在两个#之间输入的自然语言指令，并参考/的分词结果，在网页中规划（输出）一条动作序列。#请/搜索/：/当事人/是/牛晨轩/的/律师/贺涵/易/的/最近/五个/月/的/上海市/金融/法院/的/裁定书#', '检索#按钮**点击#；全文检索#输入框*牛晨轩*输入#；法院名称#输入框*上海市金融法院*输入#；裁判日期_1#日

In [6]:
###### 对B榜数据进行预测
submit_predictions = []
pattern = r"(.*?)#(\w+)\*(.*?)\*(\w+)#"
prompt = "根据用户在两个#之间输入的自然语言指令, 并参考使用/分隔的分词结果,在网页中规划（输出）一条动作序列。#{0}#"


for idx in range(len(instruction_testB)):
    page_predictions = []
    for instruction in instruction_testB[idx]['instruction']:   
        prediction = {}
        data_dict = {}
        prediction['instruction'] = instruction
        pred_data = prompt.format(tokenized(instruction))
        model_out = model.chat(tokenizer,pred_data , history=[])
        output = model_out[0].replace(";", "；") 
        datas = output.split("；")
        for item in datas:
            try:
                matches = re.findall(pattern, item)
                key = matches[0][0]
                dom_type = matches[0][1]
                value = matches[0][2]
                action = matches[0][3]
                data_dict[key] = {"dom_type": dom_type, "value":value, "action": action}
                    
            except:     
                print("{}-{}".format(instruction_testB[idx]['page_source'], model_out))
                continue
                
                   
        prediction['key-value'] = data_dict         
        page_predictions.append(prediction)

        
                                                                     
    pag_source = instruction_testA[idx]['page_source']
    submit_predictions.append({
        'page_source': pag_source, 
        'instruction_detail': page_predictions
    })
    print(pag_source+"已完成")


# 生成提交结果
with open('submission.json', 'w') as up:
    json.dump(submit_predictions, up)

qq音乐已完成
上交所已完成
中国人民银行已完成
中国法律服务网已完成
中国裁判文书网已完成
中国证券投资基金业协会已完成
中国银行保险监督管理委员会已完成
交通安全综合服务管理平台已完成
企查查企业查询-('查询#按钮**点击#；请输入企业名、人名、产品名，或地址电话/经营范围等#输入框*商汤*输入#；发源地#下拉框**点击#；四川省#下拉框选项**点击#；加拿大#下拉框选项**点击#；印度#下拉框选项**点击#；三字三字三字下拉框**点击#；资本市场#下拉框**点击#；对新三板、但未上市的企业#下拉框选项**点击#；注册资本#下拉框**点击#;200-500万#下拉框选项**点击#；纳税信用#下拉框**点击#；非A级#下拉框选项**点击#；有小程序#下拉框**点击#', [("根据用户在两个#之间输入的自然语言指令, 并参考使用/分隔的分词结果,在网页中规划（输出）一条动作序列。#我要/查询/与/'/商汤/'/有关/,/非/A/级/纳税/信用/,/新/三板/、/未上市/,/有/小/程序/,/注册资本/200/-/500/万/,/加拿大/地区/、/印度/地区/,/四川省/,/属于/房产/租赁/行业/的/企业/信息#", '查询#按钮**点击#；请输入企业名、人名、产品名，或地址电话/经营范围等#输入框*商汤*输入#；发源地#下拉框**点击#；四川省#下拉框选项**点击#；加拿大#下拉框选项**点击#；印度#下拉框选项**点击#；三字三字三字下拉框**点击#；资本市场#下拉框**点击#；对新三板、但未上市的企业#下拉框选项**点击#；注册资本#下拉框**点击#;200-500万#下拉框选项**点击#；纳税信用#下拉框**点击#；非A级#下拉框选项**点击#；有小程序#下拉框**点击#')])
企查查企业查询已完成
卫健委信用信息网_医已完成
卫健委信用信息网_护已完成
国家企业信用信息公示系统已完成
国家统计局-('开始搜索#按钮**点击#；包含以下全部的关键词#输入框*人口普查、公报*输入#；包含以下任意一个关键词#输入框*““输入#；不包含以下关键词#输入框*连续五年“输入#；一周内#单选框**点击#；显示条数#下拉框**点击#;5#下拉框选项**点击#', [('根据用户在两个#之间输入的自然语言指令, 并参考使用/分隔的分词结果,在网页中规划（输出）一条动作

In [ ]:
!zip submission.json.zip submission.json

In [3]:
result = json.load(open('./submission.json'))

In [4]:
result[0]

{'page_source': 'qq音乐',
 'instruction_detail': [{'instruction': '查询以下内容：韩国地区的歌手列表',
   'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '韩国': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '请查询欧美地区的歌手列表',
   'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '欧美': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '查找出电子风格的歌手列表',
   'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '电子': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '查找韩国地区的歌手列表',
   'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '韩国': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '查找女性的歌手列表',
   'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '女': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '请查找摇滚风格的歌手列表',
   'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '摇滚':

In [5]:
result[1]

{'page_source': '上交所',
 'instruction_detail': [{'instruction': '检索一下主板A股的信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '板块': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '主板A股': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '查一下主板B股的信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '板块': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '主板B股': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '请检索中国国贸的信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '证券代码或简称': {'dom_type': '输入框', 'value': '中国国贸', 'action': '输入'}}},
  {'instruction': '请查一下证券代码是600004的信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '证券代码或简称': {'dom_type': '输入框', 'value': '600004', 'action': '输入'}}},
  {'instruction': '请查询中国国贸的信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '证券代码或简称': {'dom_type': '输

In [6]:
result[2]

{'page_source': '中国人民银行',
 'instruction_detail': [{'instruction': '查一查包含全部关键词如”股票涨势“的查询结果',
   'key-value': {'高级搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '包含以下全部的关键词': {'dom_type': '输入框', 'value': '股票涨势', 'action': '输入'}}},
  {'instruction': '查找包含全部关键词如”征信“的查询结果',
   'key-value': {'高级搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '包含以下全部的关键词': {'dom_type': '输入框', 'value': '征信', 'action': '输入'}}},
  {'instruction': '请查找包含全部关键词如”反洗钱“的查询结果',
   'key-value': {'高级搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '包含以下全部的关键词': {'dom_type': '输入框', 'value': '反洗钱', 'action': '输入'}}},
  {'instruction': '查找包含全部关键词如”反洗钱“的查询结果',
   'key-value': {'高级搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '包含以下全部的关键词': {'dom_type': '输入框', 'value': '反洗钱', 'action': '输入'}}},
  {'instruction': '我要查询包含全部关键词如”国际贸易“的查询结果',
   'key-value': {'高级搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '包含以下全部的关键词': {'dom_type': '输入框', 'value': '国际贸易', 'action': '输入'}}},
 

In [7]:
result[3]

{'page_source': '中国法律服务网',
 'instruction_detail': [{'instruction': '查一下多禾律师事务所的信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '请输入律所关键字': {'dom_type': '输入框', 'value': '多禾律师事务所', 'action': '输入'}}},
  {'instruction': '请查询金台律师事务所的信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '请输入律所关键字': {'dom_type': '输入框', 'value': '金台律师事务所', 'action': '输入'}}},
  {'instruction': '查找江西省全部的律师事务所',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '请选择': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '江西省': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '查找出上海市全部的律师事务所',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '请选择': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '上海市': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '请查一下普光律师事务所的信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '请输入律所关键

In [8]:
result[4]

{'page_source': '中国裁判文书网',
 'instruction_detail': [{'instruction': '请查找所有的调解书',
   'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '全文检索': {'dom_type': '输入框', 'value': '调解书', 'action': '输入'},
    '高级检索': {'dom_type': '按钮', 'value': '', 'action': '点击'}}},
  {'instruction': '请搜索广东省深圳市中级人民法院的文书',
   'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '法院名称': {'dom_type': '输入框', 'value': '广东省深圳市中级人民法院', 'action': '输入'},
    '高级检索': {'dom_type': '按钮', 'value': '', 'action': '点击'}}},
  {'instruction': '请查一下国际司法协助案件文书',
   'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '案件类型': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '国际司法协助案件': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'},
    '高级检索': {'dom_type': '按钮', 'value': '', 'action': '点击'}}},
  {'instruction': '请搜索当事人是凌孟夏的文书',
   'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '当事人': {'dom_type': '输入框', 'value': '凌孟夏', 'action': '输入'},

In [9]:
result[5]

{'page_source': '中国证券投资基金业协会',
 'instruction_detail': [{'instruction': '请查一下证券公司、保险公司子公司和公募基金管理公司子公司的信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '机构类型': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '证券公司': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'},
    '设计公司': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'},
    ' livestream': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'},
    '子公司': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '保险公司子公司': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'},
    '公募基金管理公司子公司': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '查一查证券公司私募基金子公司和证券公司资管子公司的信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '证券代号或简称': {'dom_type': '输入框', 'value': '证券公司私募基金公司', 'action': '输入'},
    '公司名称': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '分类': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '证券公司私募基金公司': {'dom_type': '下拉框选项', 'va

In [10]:
result[6]

{'page_source': '中国银行保险监督管理委员会',
 'instruction_detail': [{'instruction': '查找出属于专属机构(电销-省分)层级的机构信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '机构层级': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '专属机构(电销-省分)': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '查询属于省级分公司层级的机构信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '机构层级': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '省级分公司': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '查一查受监管于辽宁银保监局的机构信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '监管机构_2': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '辽宁银保监局': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '我要查询机构类型为再保险的机构信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '机构类型_1': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '再保险': {'dom_type': '下拉框选项', 'v

In [11]:
result[7]

{'page_source': '交通安全综合服务管理平台',
 'instruction_detail': [{'instruction': '查找出决定书编号为1114002800140830的公示信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '决定书编号': {'dom_type': '输入框',
     'value': '1114002800140830',
     'action': '输入'}}},
  {'instruction': '请查询中国人寿企业的公示信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '企业名称': {'dom_type': '输入框', 'value': '人寿', 'action': '输入'}}},
  {'instruction': '请查一下决定书编号为1102002800178260的公示信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '决定书编号': {'dom_type': '输入框',
     'value': '1102002800178260',
     'action': '输入'}}},
  {'instruction': '请搜索：社会信用代码为911101117934428954的公示信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '社会信用代码': {'dom_type': '输入框',
     'value': '911101117934428954',
     'action': '输入'},
    '公示时间_1': {'dom_type': '日期输入框', 'value': '2022-03-15', 'action': '输入'},
    '公示时间_2': {'dom_type': '日期输入框', 'value': 

In [12]:
result[8]

{'page_source': '企查查企业查询',
 'instruction_detail': [{'instruction': "我想知道与'中国石油'有关的企业信息",
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '请输入企业名、人名、产品名，或地址电话/经营范围等': {'dom_type': '输入框',
     'value': '中国石油',
     'action': '输入'}}},
  {'instruction': "查找一下与'太平洋保险'有关的企业信息",
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '请输入企业名、人名、产品名，或地址电话/经营范围等': {'dom_type': '输入框',
     'value': '太平洋保险',
     'action': '输入'}}},
  {'instruction': "查询与'张一鸣'有关的企业信息",
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '请输入企业名、人名、产品名，或地址电话/经营范围等': {'dom_type': '输入框',
     'value': '张一鸣',
     'action': '输入'}}},
  {'instruction': "查一查与'阿里巴巴'有关的企业信息",
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '请输入企业名、人名、产品名，或地址电话/经营范围等': {'dom_type': '输入框',
     'value': '阿里巴巴',
     'action': '输入'}}},
  {'instruction': "请查一下与'中国石油'有关的企业信息",
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'}

In [13]:
result[9]

{'page_source': '卫健委信用信息网_医',
 'instruction_detail': [{'instruction': '查一下上海市第二人民医院的医师信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '执业地点': {'dom_type': '输入框', 'value': '上海市第二人民医院', 'action': '输入'}}},
  {'instruction': '查找韦希的医师信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '医师姓名': {'dom_type': '输入框', 'value': '韦希', 'action': '输入'}}},
  {'instruction': '请搜索韦希的医师信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '医师姓名': {'dom_type': '输入框', 'value': '韦希', 'action': '输入'}}},
  {'instruction': '请查一下上海康新医院的医师信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '执业地点': {'dom_type': '输入框', 'value': '上海康新医院', 'action': '输入'}}},
  {'instruction': '查找出执业类别为中西医结合的医师信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '执业类别': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '中西医结合': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'i

In [14]:
result[10]

{'page_source': '卫健委信用信息网_护',
 'instruction_detail': [{'instruction': '查找出上海市黄浦区中西医结合医院的护士信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '执业地点': {'dom_type': '输入框', 'value': '上海市黄浦区中西医结合医院', 'action': '输入'}}},
  {'instruction': '检索一下南京东路地段医院的护士信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '执业地点': {'dom_type': '输入框', 'value': '南京东路地段医院', 'action': '输入'}}},
  {'instruction': '查一下郑念雁的护士信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '护士姓名': {'dom_type': '输入框', 'value': '郑念雁', 'action': '输入'}}},
  {'instruction': '请查找上海康新医院的护士信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '执业地点': {'dom_type': '输入框', 'value': '上海康新医院', 'action': '输入'}}},
  {'instruction': '搜索宗彦的护士信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '护士姓名': {'dom_type': '输入框', 'value': '宗彦', 'action': '输入'}}},
  {'instruction': '请搜索上海申江医院的护士信息',
   'key-value': {'

In [15]:
result[11]

{'page_source': '国家企业信用信息公示系统',
 'instruction_detail': [{'instruction': '请查询在黑龙江的企业信息',
   'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '黑龙江': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '请搜索：经营异常名录',
   'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '经营异常名录': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '查询在福建的企业信息',
   'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '福建': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '请检索有行政处罚的企业信息',
   'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '有行政处罚': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '查询严重违法失信名单',
   'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '严重违法失信名单': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '请查找企业状态为歇业的企业信息',
   'key-value': {'搜索': {'dom_type': '按钮', 'value': '', 'acti

In [16]:
result[12]

{'page_source': '国家统计局',
 'instruction_detail': [{'instruction': '查找一下包含以下全部的关键词如”GDP、近五年“的查询结果',
   'key-value': {'开始搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '包含以下全部的关键词': {'dom_type': '输入框', 'value': 'GDP、近五年', 'action': '输入'}}},
  {'instruction': '查找出包含以下全部的关键词如”GDP、近五年“的查询结果',
   'key-value': {'开始搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '包含以下全部的关键词': {'dom_type': '输入框', 'value': 'GDP、近五年', 'action': '输入'}}},
  {'instruction': '查询以下内容：包含以下全部的关键词如”平均工资“的查询结果',
   'key-value': {'开始搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '包含以下全部的关键词': {'dom_type': '输入框', 'value': '平均工资', 'action': '输入'}}},
  {'instruction': '请查找包含以下全部的关键词如”CPI“的查询结果',
   'key-value': {'开始搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '包含以下全部的关键词': {'dom_type': '输入框', 'value': 'CPI', 'action': '输入'}}},
  {'instruction': '查一下包含以下全部的关键词如”人口普查、公报“的查询结果',
   'key-value': {'开始搜索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '包含以下全部的关键词': {'dom_type': '输入

In [17]:
result[13]

{'page_source': '学信网',
 'instruction_detail': [{'instruction': '查一查军事学学硕的硕士招生情况',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '门类类别': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '11军事学': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '我要查询专业学位的硕士招生情况',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '门类类别': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '00专业学位': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '查找出非全日制的硕士招生情况',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '学习方式': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '非全日制': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '帮我查一下全日制的硕士招生情况',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '学习方式': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '全日制': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'in

In [18]:
result[14]

{'page_source': '安居客',
 'instruction_detail': [{'instruction': '查找出嘉定区的二手房信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '区域': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '嘉定': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '查一下不限房型的二手房信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '不限': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '请查询精装修的二手房信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '精装修': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '查询以下内容：正西朝向的二手房信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '朝向': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '西': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '请查询一室的二手房信息',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '一室': {'dom_type': '单选框', 'value': '', 'acti

In [19]:
result[15]

{'page_source': '巨潮资讯',
 'instruction_detail': [{'instruction': '查找深沪京的三季报和澄清致歉公告',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '深沪京': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '分类': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '三季报': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'},
    '澄清致歉': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'},
    '确认': {'dom_type': '按钮', 'value': '', 'action': '点击'}}},
  {'instruction': '请搜索2021年1月1日到2023年1月1日的深沪京的中介报告、董事会和股东大会公告',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '2021-01-01': {'dom_type': '日期输入框', 'value': '', 'action': '输入'},
    '2023-01-01': {'dom_type': '日期输入框', 'value': '', 'action': '输入'},
    '深沪京': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '分类': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '中介报告': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'},
    '董事会': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'},
    '股东大会': {

In [20]:
result[16]

{'page_source': '火车票12306',
 'instruction_detail': [{'instruction': '请搜索：济南的天气状况',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '区域': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '济南': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '请查一下兰州的天气状况',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '地点': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '兰州': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '查询长春的天气状况',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '城市': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '长春': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '请查询长沙的天气情况',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '地点': {'dom_type': '下拉框', 'value': '', 'action': '点击'},
    '长沙': {'dom_type': '下拉框选项', 'value': '', 'action': '点击'}}},
  {'instruction': '请搜索：太原的天气状况',
   '

In [21]:
result[17]

{'page_source': '猫眼电影',
 'instruction_detail': [{'instruction': '查一查其他类的电影',
   'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '其他': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '查找一下灾难类的电影',
   'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '灾难': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '查找一下按时间排序的电影',
   'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '按时间排序': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '查一查2018年的电影',
   'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '2018': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '查找一下泰国地区的电影',
   'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '泰国': {'dom_type': '单选框', 'value': '', 'action': '点击'}}},
  {'instruction': '查找出按评价排序的电影',
   'key-value': {'检索': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '按评价排序'

In [22]:
result[18]

{'page_source': '顺丰快递查收寄标准',
 'instruction_detail': [{'instruction': '查询能否以个人身份把书籍从天津市增城区寄往加拿大Zengcheng，其中收件人身份是个人',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '始发地': {'dom_type': '输入框', 'value': '天津市/天津市/增城区', 'action': '输入'},
    '目的地': {'dom_type': '输入框', 'value': '加拿大/Zengcheng', 'action': '输入'},
    '个人_1': {'dom_type': '单选框', 'value': '', 'action': '点击'},
    '个人_2': {'dom_type': '单选框', 'value': '', 'action': '点击'},
    '物品名称': {'dom_type': '输入框', 'value': '书籍', 'action': '输入'}}},
  {'instruction': '请检索是否可以把电脑从浙江省嘉兴市增城区寄到加拿大Zengcheng，其中寄件人身份是公司，收件人身份是公司',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '始发地': {'dom_type': '输入框', 'value': '浙江省/嘉兴市/增城区', 'action': '输入'},
    '目的地': {'dom_type': '输入框', 'value': '加拿大/Zengcheng', 'action': '输入'},
    '公司_1': {'dom_type': '单选框', 'value': '', 'action': '点击'},
    '公司_2': {'dom_type': '单选框', 'value': '', 'action': '点击'},
    '物品名称': {'dom_type': '输入框', 'value': '电脑', 'action': 

In [23]:
result[19]

{'page_source': '顺丰快递查运费时效',
 'instruction_detail': [{'instruction': '请检索从四川省成都市从化区寄到美国Yuexiu的快递时效，其中寄件时间是2023-01-10上午09:12:14，包裹重量为0.6kg',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '始发地': {'dom_type': '输入框', 'value': '四川省-成都市-从化区', 'action': '输入'},
    '目的地': {'dom_type': '输入框', 'value': '美国-Yuexiu', 'action': '输入'},
    '寄件时间': {'dom_type': '输入框',
     'value': '2023-01-10 09:12:14',
     'action': '输入'},
    '包裹重量': {'dom_type': '输入框', 'value': '0.600', 'action': '输入'}}},
  {'instruction': '查找从香港荔湾区发往加拿大Zengcheng的快递运费，其中包裹重量为0.8kg，寄件时间是2023-01-17下午12:23:45',
   'key-value': {'查询': {'dom_type': '按钮', 'value': '', 'action': '点击'},
    '始发地': {'dom_type': '输入框', 'value': '香港-荔湾区', 'action': '输入'},
    '目的地': {'dom_type': '输入框', 'value': '加拿大-Zengcheng', 'action': '输入'},
    '寄件时间': {'dom_type': '输入框',
     'value': '2023-01-17 12:23:45',
     'action': '输入'},
    '包裹重量': {'dom_type': '输入框', 'value': '0.800', 'action': '输入'}}},
  {'instruction': '请检索从福建